# APS 3 - Aula 20
---

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from funcoesTermosol import *
from app_utils import *

In [2]:
num_nos, matriz_nos, num_membros, matriz_incidencia, num_cargas, v_carregamento, num_rest, v_rest = importa("entrada.xlsx")

In [3]:
conectividade_global = conec_global_T(num_nos, matriz_incidencia)

In [4]:
matriz_dos_membros = np.dot(matriz_nos, conectividade_global)

In [5]:
KG = matriz_global(num_membros, matriz_incidencia, matriz_nos, matriz_dos_membros)

In [6]:
Pg = np.array([0, 150, -100]).T # Condição de contorno

In [7]:
KG_solucao = MR_para_solucao(KG)

In [8]:
u = calcula_deslocamentos(KG_solucao, Pg)

In [9]:
u_completo = np.array([0, u[0], 0, 0, u[1], u[2]])

In [10]:
R1x = calculate_force(KG, u_completo, 0) 
R2x = calculate_force(KG, u_completo, 2)
R2y = calculate_force(KG, u_completo, 3)

In [11]:
for i in range(num_membros):
    tensao, deformacao = tensao_e_deformacao(i+1, num_membros, u_completo, matriz_incidencia, matriz_nos)
    print("Membro {0}: tensao: {1} | deformacao: {2}".format(i+1, tensao, deformacao))

TypeError: tensao_e_deformacao() takes 3 positional arguments but 5 were given